# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [ ]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_7월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(14192, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/kittiepie/222047759528,7月 31日: 칠월의 마지막 밤은 매미 울음 속에,2020.07.31.
1,https://blog.naver.com/minghae00/222047758389,여름휴가 1일차:고모리저수지/경기도잣나무푸른숲/축령산...,2020.07.31.
2,https://blog.naver.com/chicbook/222047758534,한우리독서토론논술 초등부 8월 프로그램,2020.07.31.
3,https://blog.naver.com/wsc1025/222047756291,"아이와 구한말의 세 주인공들을 만나다! 고종, 흥선대원군...",2020.07.31.
4,https://blog.naver.com/misspark365/222047750066,강진 여행 1박2일 일정 + 나홀로 야간여행,2020.07.31.
...,...,...,...
14187,https://blog.naver.com/namsan_couple/222016467737,강릉 여행 경포대 가볼 곳 허균생가 가족과 함께 가기 좋은 곳,2020.07.01.
14188,https://blog.naver.com/correctgame/222017310122,1292. 2017.7.16. 대한민국 경상북도 경주시...,2020.07.01.
14189,https://blog.naver.com/jubalry/222016833799,넷마블 - 중국 관련주(게임주) & BTS(방탄소년단) 관련주,2020.07.01.
14190,https://blog.naver.com/dalloway7374/222017177943,김용남展 - 삶의 미학 - 심화,2020.07.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


14192


In [ ]:
# df = pd.DataFrame(columns = {'url','본문'})
# df.to_csv("./나무문화/2020/나무문화_본문_2020_7월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')

In [5]:
df = pd.read_csv("./나무문화/2020/나무문화_본문_2020_7월.csv", encoding='UTF-8')
df

,본문,url
0,1. 이번 7월은 허연의 칠월을 읽지 않았고 루카 구아다니노의 아이엠러브도 보지 않...,https://blog.naver.com/kittiepie/222047759528
1,2020년 7월 31일\n여름휴가 1일차\n일정: 포천 고모리 욕쟁이할머니집 점심 ...,https://blog.naver.com/minghae00/222047758389
2,생각하는 나무 어휘리더 1 (초등1)\n1차시 자신만만 읽기 • 쓰기\n《말짱 도루...,https://blog.naver.com/chicbook/222047758534
3,"반갑습니다. 오늘은 인사동 주변 볼거리를 소개합니다. 바로 운현궁, 천도교중앙대교당...",https://blog.naver.com/wsc1025/222047756291
4,"강진 여행 1박2일 일정 + 나홀로아간여행\n글, 사진, 영상 ⓒ박매력\n대한민국 ...",https://blog.naver.com/misspark365/222047750066
...,...,...
2668,"높이 2,744m(중국측 발표는 2,749.6m)이다. 활화산(活火山)으로 북위 4...",https://blog.naver.com/younskin1610/222042096214
2669,변기에 빠진 세계사\n저자\n이영숙\n출판\n자음과모음\n발매\n2020.07.10...,https://blog.naver.com/ddamddi79/222042094787
2670,"서울에서 귀한 손님들이 오셨다.\n동학콘텐츠연구회 선생님들이\n고창, 정읍을 거쳐 ...",https://blog.naver.com/arabianna/222042090163
2671,마지막 날이에요.\n소소한 뜰 문 밖으로 보이는 풍경이... 오늘따라 더 그림.\n...,https://blog.naver.com/rubyrhee2/222042087104


In [6]:
len(df)

2673

In [ ]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_7월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


In [ ]:
df

In [ ]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))
